In [135]:
import os
import glob
import yaml
import pandas as pd

cwd = os.getcwd()

In [136]:
with open("config.yaml", 'r', encoding='utf-8') as stream:
    config = yaml.safe_load(stream)
config

{'curve': {'import': None,
  'columns_old': ['Date (YYYY-MM-DD as UTC)',
   'Merchant',
   'Txn Amount (Funding Card)',
   'Card Name',
   'Category'],
  'columns_new': ['Day', 'Expense name', 'Amount', 'Bank', 'Comment'],
  'Day': '%Y-%m-%d'},
 'ing': {'import': {'sep': ';'},
  'columns_old': ['Date', 'Bénéficiaire', 'Montant', 'Bank', 'Communication'],
  'columns_new': ['Day', 'Expense name', 'Amount', 'Bank', 'Comment'],
  'Day': '%d/%m/%Y'},
 'n26': {'import': None,
  'columns_old': ['Data',
   'Beneficiario',
   'Importo (EURO)',
   'Bank',
   'Riferimento pagamento'],
  'columns_new': ['Day', 'Expense name', 'Amount', 'Bank', 'Comment'],
  'Remove': ['CRV*'],
  'Day': '%Y-%m-%d'},
 'revolut': {'import': None,
  'columns_old': ['Started Date', 'Description', 'Amount', 'Bank', 'Type'],
  'columns_new': ['Day', 'Expense name', 'Amount', 'Bank', 'Comment'],
  'Remove': ['Crv*'],
  'Day': '%Y-%m-%d %H:%M:%S'}}

In [137]:
folders_data = os.listdir('data')
folders_data

['curve', 'ing', 'n26', 'revolut']

In [138]:
file_data = {folder: max(glob.glob(os.path.join(cwd, 'data', folder, '*')), key=os.path.getctime) for folder in folders_data}
file_data

{'curve': 'c:\\Users\\giamp\\Dropbox\\GitHub\\import-bank-details\\data\\curve\\export.csv',
 'ing': 'c:\\Users\\giamp\\Dropbox\\GitHub\\import-bank-details\\data\\ing\\export_LU980141271390000000_20220301_20220618.xlsx',
 'n26': 'c:\\Users\\giamp\\Dropbox\\GitHub\\import-bank-details\\data\\n26\\n26-csv-transactions (2).csv',
 'revolut': 'c:\\Users\\giamp\\Dropbox\\GitHub\\import-bank-details\\data\\revolut\\account-statement_2022-03-01_2022-06-18_it_b51eb4.csv'}

In [145]:
df = None

for file_key, file_value in file_data.items():
    try:
        df_temp = pd.read_csv(file_value, **config[file_key]['import'])
    except TypeError:
        df_temp = pd.read_csv(file_value)
    except UnicodeDecodeError:
        df_temp = pd.read_excel(file_value)
    
    if 'Bank' in config[file_key]['columns_old']:
        df_temp['Bank'] = file_key
    df_temp = df_temp[config[file_key]['columns_old']]
    df_temp = df_temp.rename(columns=dict(zip(config[file_key]['columns_old'], config[file_key]['columns_new'])))
    
    try:
        df_temp = df_temp[~df_temp['Expense name'].str.contains('|'.join(config[file_key]['Remove']))]
    except KeyError:
        pass

    df_temp['Day'] = pd.to_datetime(df_temp['Day'], format=config[file_key]['Day'])

    
    df = pd.concat([df, df_temp], ignore_index=True)
df['Day'] = df['Day'].dt.round('D')

df = df.sort_values(by=['Day', 'Expense name', 'Amount'])

df['Amount'] = -df['Amount']

filename = f"{df['Day'].max().strftime('%Y-%m-%d')}_{'-'.join(config.keys())}.xlsx"
df.to_excel(os.path.join('output', filename), index=False)

df

,Day,Expense name,Amount,Bank,Comment
589,2020-08-21,Ref: Giampaolo C.,-500.00,Curve Cash,NaN
588,2020-09-05,Auchan Cloche D'or,22.55,Revolut Savings,Groceries
587,2020-09-05,Curve Cash: Auchan Cloche D'or,-23.00,Curve Cash,NaN
586,2020-09-06,Goedert Autocenter,14.30,N26 Giamp,Shopping
585,2020-09-20,Amzn Mktp De*fp47m5pw5,20.86,Revolut Savings,Shopping
...,...,...,...,...,...
2,2022-06-16,Amzn Mktp UK,15.55,Revolut Groceries + Out,Shopping
1,2022-06-17,Lidl,50.15,Revolut Groceries + Out,Groceries
724,2022-06-17,Pharmacie De La Liberte,1.00,revolut,CARD_PAYMENT
0,2022-06-17,Rest Sushi Shop Kirchb,33.00,Revolut Groceries + Out,Eating Out


In [101]:
df['Day'] = df['Day'].dt.round('D')

In [144]:
name = 'ing'
file_key, file_value = name, file_data[name]
try:
    df_temp = pd.read_csv(file_value, **config[file_key]['import'])
except TypeError:
    df_temp = pd.read_csv(file_value)
except UnicodeDecodeError:
    df_temp = pd.read_excel(file_value)
df_temp

,#,Référence,Opération,Date,Date valeur,Date comptable,Montant,Devise,Bénéficiaire,Compte bénéficiaire,Communication,Carte,Numéro,Alias,Contrevaleur,Devise.1,Ville,Pays
0,1,OP06026244,180Savings + Gifts + Clothes,2022-03-01,2022-03-01,2022-03-01,-230.00,EUR,Giampaolo Casolla,LT753250003855709822,180Savings + Gifts + Clothes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,OP06016277,Rent Apartment 57 Rue de Neudorf,2022-03-01,2022-03-01,2022-03-01,-1550.00,EUR,M. DESPONTIN Philippe,LU710099780000451419,Rent Apartment 57 Rue de Neudorf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,OP06024434,Investing + Guilt-free Spending,2022-03-01,2022-03-01,2022-03-01,-1783.00,EUR,Giampaolo Casolla,DE31100110012622391346,Investing + Guilt-free Spending,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,PC330JZ8XK,Virement Européen DESPONTIN PHILIPPE ANDRE rem...,2022-03-03,2022-03-03,2022-03-03,546.04,EUR,DESPONTIN PHILIPPE ANDRE,LU710099780000451419,remboursement surplus de charges,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,PC340K24D2,Virement Européen AMAZON EU S.A R.L. FCS001066...,2022-03-04,2022-03-04,2022-03-04,95.00,EUR,AMAZON EU S.A R.L.,IE04BOFA99006156667038,FCS001066137002 PAYMENT-NUM 1 97504845,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,PC330JZ0T7,Domiciliation Européenne BASIC FIT LUXEMBOURG ...,2022-03-04,2022-03-04,2022-03-04,-39.99,EUR,BASIC FIT LUXEMBOURG,NL47ABNA0539927031,06-03-2022 / 02-04-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,9780623357,Giampaolo Casolla,2022-03-04,2022-03-04,2022-03-04,-95.00,EUR,Giampaolo Casolla,DE31100110012622391346,Partigiano reimbursement,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,7750670024,RETRAIT SELF'BANK DU 09.03.22 CARTE NO 359968,2022-03-09,2022-03-09,2022-03-09,-160.00,EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,PC3L0KQ44K,Domiciliation Européenne PROXIMUS LUXEMBOURG S...,2022-03-21,2022-03-21,2022-03-21,-57.99,EUR,PROXIMUS LUXEMBOURG S.A.,LU830030047073620000,Tango/27441327/27441327220201,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,9670773161,VIREMENT(S) CASOLLA GIAMPAO,2022-03-21,2022-03-21,2022-03-21,-229.00,EUR,MR CASOLLA GIAMPAOLO,LU110141371390010000,Booking London,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
if 'Bank' in config[file_key]['columns_old']:
    df_temp['Bank'] = file_key
df_temp = df_temp[config[file_key]['columns_old']]
df_temp = df_temp.rename(columns=dict(zip(config[file_key]['columns_old'], config[file_key]['columns_new'])))
try:
    df_temp = df_temp[~df_temp['Expense name'].str.contains('|'.join(config[file_key]['Remove']))]
except KeyError:
    pass
df_temp['Day'] = pd.to_datetime(df_temp['Day'], format=config[file_key]['Day'])
df_temp

,Day,Expense name,Amount,Bank,Comment
0,2022-03-01 10:27:47,Eurest Intesa San Paolo,-1.45,revolut,CARD_PAYMENT
1,2022-03-02 12:24:41,Auchan Cloche D'or,-4.48,revolut,CARD_PAYMENT
12,2022-03-11 17:03:03,Auchan Cloche D'or,-7.02,revolut,CARD_PAYMENT
13,2022-03-11 17:29:38,Pharmacie De La Cloche D',-0.78,revolut,CARD_PAYMENT
14,2022-03-12 13:24:51,Sumup *pittige Dames Foo,-10.00,revolut,CARD_PAYMENT
23,2022-03-15 16:38:41,Auchan Cloche D'or,-6.75,revolut,CARD_PAYMENT
24,2022-03-16 18:54:54,Auchan Cloche D'or,-8.70,revolut,CARD_PAYMENT
34,2022-03-30 08:04:31,Payment from Clementina Uccello,850.00,revolut,TOPUP
54,2022-04-04 11:24:07,Auchan Cloche D'or,-16.97,revolut,CARD_PAYMENT
60,2022-04-08 22:31:38,To Clementina Uccello,-3.80,revolut,TRANSFER


In [ ]:
try:
    df.append(df_temp, ignore_index=True)
except:
    df = df_temp

In [88]:
df

,Day,Expense name,Amount,Bank,Comment
0,2022-06-12,Amzn Mktp De*2c4zk9h14,-28.41,Revolut Groceries + Out,Shopping
1,2022-06-12,Amazon,-13.76,Revolut Groceries + Out,Shopping
2,2022-06-12,Tapas Street,-3.00,Revolut Groceries + Out,Eating Out
3,2022-06-12,Tapas Street,-9.60,Revolut Groceries + Out,Eating Out
4,2022-06-12,Ville De Remich : Pisc,-14.00,Revolut Groceries + Out,Entertainment
...,...,...,...,...,...
580,2020-09-20,Amzn Mktp De*fp47m5pw5,-20.86,Revolut Savings,Shopping
581,2020-09-06,Goedert Autocenter,-14.30,N26 Giamp,Shopping
582,2020-09-05,Curve Cash: Auchan Cloche D'or,23.00,Curve Cash,NaN
583,2020-09-05,Auchan Cloche D'or,-22.55,Revolut Savings,Groceries
